In [3]:
import os
import sys
import subprocess
import numpy as np
import itk
# Add the 'src' folder to the Python path
sys.path.append(os.path.abspath('../'))
from src.utils import match_h5_files_by_channels, setup_logging, collect_region_stats_paths
from src.registration import register_and_transform
from src.stats import compute_region_stats
import logging

# single animal

In [2]:
animal = 'ANM555600_PSD-HT_iDISCO'
base_dir = '/nrs/spruston/Boaz/I2/2024-09-09_Compare_iDISCO_EZCleat_THF'

In [11]:
animal = 'ANM550749_left_JF552'
base_dir = '/nrs/spruston/Boaz/I2/2024-09-19_iDISCO_CalibrationBrains'

In [12]:
# Set the parameters manually here

fx_path = '/nrs/spruston/Boaz/I2/atlas10_hemi.tif'
param_files_dir = '/nrs/spruston/Boaz/I2/itk'
annotation_path = '/nrs/spruston/Boaz/I2/annotation_10_hemi.nii'

In [4]:
setup_logging(base_dir, animal)
# Load the fixed image (fx)
logging.info(f"Loading fixed image (fx) from {fx_path}.")
fx = itk.imread(fx_path, pixel_type=itk.US)

# Load the parameter files
param_files = [
    os.path.join(param_files_dir, 'Order1_Par0000affine.txt'),
    os.path.join(param_files_dir, 'Order3_Par0000bspline.txt'),
    os.path.join(param_files_dir, 'Order4_Par0000bspline.txt'),
    os.path.join(param_files_dir, 'Order5_Par0000bspline.txt')
]
logging.info(f"Loaded parameter files from {param_files_dir}.")



2024-09-27 11:58:45 - INFO - Logging initialized for animal processing.
TIFFReadDirectory: Warning, Unknown field with tag 50838 (0xc696) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 50839 (0xc697) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 50838 (0xc696) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 50839 (0xc697) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 50838 (0xc696) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 50839 (0xc697) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 50838 (0xc696) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 50839 (0xc697) encountered.


In [5]:
# Load the annotation volume
logging.info(f"Loading annotation volume from {annotation_path}.")
itk_annotation = itk.imread(annotation_path, itk.ULL)
annotation_np = itk.array_view_from_image(itk_annotation)
annotation_np.dtype, annotation_np.shape


(dtype('uint64'), (800, 1320, 658))

In [6]:
# Match H5 files by channels for all animals
logging.info(f"Matching H5 files in {base_dir}.")
animals_files = match_h5_files_by_channels(base_dir)

In [7]:
animals_files

{'ANM555600_PSD-HT_iDISCO': {'ch0': '/nrs/spruston/Boaz/I2/2024-09-09_Compare_iDISCO_EZCleat_THF/ANM555600_PSD-HT_iDISCO/uni_tp-0_ch-0_st-0-x00-y00_obj-right_cam-long_etc.lux.h5',
  'ch1': '/nrs/spruston/Boaz/I2/2024-09-09_Compare_iDISCO_EZCleat_THF/ANM555600_PSD-HT_iDISCO/uni_tp-0_ch-1_st-0-x00-y00_obj-right_cam-long_etc.lux.h5',
  'ch2': '/nrs/spruston/Boaz/I2/2024-09-09_Compare_iDISCO_EZCleat_THF/ANM555600_PSD-HT_iDISCO/uni_tp-0_ch-2_st-0-x00-y00_obj-right_cam-long_etc.lux.h5'},
 '20240916-140630_Task_77_iDISCO_C_RegisterEachChannel': {'ch0': '/nrs/spruston/Boaz/I2/2024-09-09_Compare_iDISCO_EZCleat_THF/ANM555600_PSD-HT_iDISCO/20240916-140630_Task_77_iDISCO_C_RegisterEachChannel/uni_tp-0_ch-0_st-0-x00-y00_obj-right_cam-long_etc.lux.h5',
  'ch1': '/nrs/spruston/Boaz/I2/2024-09-09_Compare_iDISCO_EZCleat_THF/ANM555600_PSD-HT_iDISCO/20240916-140630_Task_77_iDISCO_C_RegisterEachChannel/uni_tp-0_ch-1_st-0-x00-y00_obj-right_cam-long_etc.lux.h5',
  'ch2': '/nrs/spruston/Boaz/I2/2024-09-09_Co

In [8]:
# Get the files for the selected animal
files = animals_files.get(animal)
files

{'ch0': '/nrs/spruston/Boaz/I2/2024-09-09_Compare_iDISCO_EZCleat_THF/ANM555600_PSD-HT_iDISCO/uni_tp-0_ch-0_st-0-x00-y00_obj-right_cam-long_etc.lux.h5',
 'ch1': '/nrs/spruston/Boaz/I2/2024-09-09_Compare_iDISCO_EZCleat_THF/ANM555600_PSD-HT_iDISCO/uni_tp-0_ch-1_st-0-x00-y00_obj-right_cam-long_etc.lux.h5',
 'ch2': '/nrs/spruston/Boaz/I2/2024-09-09_Compare_iDISCO_EZCleat_THF/ANM555600_PSD-HT_iDISCO/uni_tp-0_ch-2_st-0-x00-y00_obj-right_cam-long_etc.lux.h5'}

In [9]:
output_dir = os.path.join(base_dir, animal, 'itk')  # Ensure logs and outputs go to /itk
# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)
logging.info(f"Using output dir: {output_dir}")

In [ ]:
# Step 1: Register and Transform
logging.info("Starting registration and transformation.")
register_and_transform(fx, files, output_dir, param_files)
logging.info("Finished registration and transformation.")

In [10]:
# Step 2: Compute Region Statistics
logging.info("Starting computation of region statistics.")
funcs = [np.mean, np.median, np.std]
num_cores = os.cpu_count()  # Automatically detect the number of cores
df_stats = compute_region_stats(files, output_dir, annotation_np, funcs, num_cores)
logging.info("Finished computation of region statistics.")

[<function mean at 0x149fc41d1eb0>, <function median at 0x149fb563dd70>, <function std at 0x149fc41d20b0>]


Processing labels: 100%|███████████████████████████████████████████████████████████| 687/687 [01:05<00:00, 10.57label/s]


Region statistics saved to /nrs/spruston/Boaz/I2/2024-09-09_Compare_iDISCO_EZCleat_THF/ANM555600_PSD-HT_iDISCO/itk/region_stats.csv


In [ ]:
# Log the location of the logs and output files
log_file = os.path.join(output_dir, f'registration_log_{animal}.txt')
logging.info(f"Processing logs and outputs are saved in: {log_file}")

# Multiple animals locally

In [4]:
animals = ['ANM549057_left_JF522','ANM550749_left_JF552', 
           'ANM550751_left_JF673', 'ANM551089_left_JF673']
base_dir = '/nrs/spruston/Boaz/I2/2024-09-19_iDISCO_CalibrationBrains'
fx_path = '/nrs/spruston/Boaz/I2/atlas10_hemi.tif'
param_files_dir = '/nrs/spruston/Boaz/I2/itk'
annotation_path = '/nrs/spruston/Boaz/I2/annotation_10_hemi.nii'

In [5]:
# Load the annotation volume
logging.info(f"Loading annotation volume from {annotation_path}.")
itk_annotation = itk.imread(annotation_path, itk.ULL)
annotation_np = itk.array_view_from_image(itk_annotation)
annotation_np.dtype, annotation_np.shape


(dtype('uint64'), (800, 1320, 658))

In [6]:
# Match H5 files by channels for all animals
logging.info(f"Matching H5 files in {base_dir}.")
animals_files = match_h5_files_by_channels(base_dir)

In [7]:
for animal in animals:
    setup_logging(base_dir, animal)
    # Load the fixed image (fx)
    logging.info(f"Loading fixed image (fx) from {fx_path}.")
    fx = itk.imread(fx_path, pixel_type=itk.US)

    # Load the parameter files
    param_files = [
        os.path.join(param_files_dir, 'Order1_Par0000affine.txt'),
        os.path.join(param_files_dir, 'Order3_Par0000bspline.txt'),
        os.path.join(param_files_dir, 'Order4_Par0000bspline.txt'),
        os.path.join(param_files_dir, 'Order5_Par0000bspline.txt')
    ]
    logging.info(f"Loaded parameter files from {param_files_dir}.")
    # Get the files for the selected animal
    files = animals_files.get(animal)
    output_dir = os.path.join(base_dir, animal, 'itk')  # Ensure logs and outputs go to /itk
    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    logging.info(f"Using output dir: {output_dir}")
    logging.info("Starting computation of region statistics.")
    funcs = [np.mean, np.median, np.std]
    num_cores = os.cpu_count()  # Automatically detect the number of cores
    df_stats = compute_region_stats(files, output_dir, annotation_np, funcs, num_cores)

INFO:ANM549057_left_JF522:Logging initialized for animal processing.
TIFFReadDirectory: Warning, Unknown field with tag 50838 (0xc696) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 50839 (0xc697) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 50838 (0xc696) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 50839 (0xc697) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 50838 (0xc696) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 50839 (0xc697) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 50838 (0xc696) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 50839 (0xc697) encountered.


[<function mean at 0x14e76d0fb870>, <function median at 0x14e76d048ef0>, <function std at 0x14e76d0fba30>]


Processing labels: 100%|███████████████████████████████████████████████████████████| 687/687 [00:58<00:00, 11.83label/s]
INFO:ANM550749_left_JF552:Logging initialized for animal processing.


Region statistics saved to /nrs/spruston/Boaz/I2/2024-09-19_iDISCO_CalibrationBrains/ANM549057_left_JF522/itk/region_stats.csv


TIFFReadDirectory: Warning, Unknown field with tag 50838 (0xc696) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 50839 (0xc697) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 50838 (0xc696) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 50839 (0xc697) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 50838 (0xc696) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 50839 (0xc697) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 50838 (0xc696) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 50839 (0xc697) encountered.


...

Region statistics saved to /nrs/spruston/Boaz/I2/2024-09-19_iDISCO_CalibrationBrains/ANM550751_left_JF673/itk/region_stats.csv


TIFFReadDirectory: Warning, Unknown field with tag 50838 (0xc696) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 50839 (0xc697) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 50838 (0xc696) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 50839 (0xc697) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 50838 (0xc696) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 50839 (0xc697) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 50838 (0xc696) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 50839 (0xc697) encountered.


[<function mean at 0x14e76d0fb870>, <function median at 0x14e76d048ef0>, <function std at 0x14e76d0fba30>]


Processing labels: 100%|███████████████████████████████████████████████████████████| 687/687 [00:58<00:00, 11.74label/s]


Region statistics saved to /nrs/spruston/Boaz/I2/2024-09-19_iDISCO_CalibrationBrains/ANM551089_left_JF673/itk/region_stats.csv


In [9]:
paths = collect_region_stats_paths('/nrs/spruston/Boaz/I2/')

In [10]:
paths

{'555600': '/nrs/spruston/Boaz/I2/2024-09-09_Compare_iDISCO_EZCleat_THF/ANM555600_PSD-HT_iDISCO/itk/region_stats.csv',
 '549057': '/nrs/spruston/Boaz/I2/2024-09-19_iDISCO_CalibrationBrains/ANM549057_left_JF522/itk/region_stats.csv',
 '550749': '/nrs/spruston/Boaz/I2/2024-09-19_iDISCO_CalibrationBrains/ANM550749_left_JF552/itk/region_stats.csv',
 '550751': '/nrs/spruston/Boaz/I2/2024-09-19_iDISCO_CalibrationBrains/ANM550751_left_JF673/itk/region_stats.csv',
 '551089': '/nrs/spruston/Boaz/I2/2024-09-19_iDISCO_CalibrationBrains/ANM551089_left_JF673/itk/region_stats.csv'}

# Multiple animals using bsub

In [22]:
# Define parameters
# base_dir = '/nrs/spruston/Boaz/I2/2024-09-19_iDISCO_CalibrationBrains'

base_dir = '/nrs/spruston/Boaz/I2/2024-09-09_Compare_iDISCO_EZCleat_THF'
fx = '/nrs/spruston/Boaz/I2/atlas10_hemi.tif'
param_files_dir = '/nrs/spruston/Boaz/I2/itk'
annotation_np = '/nrs/spruston/Boaz/I2/annotatin10_hemi.tif'
python_script = os.path.abspath('../src/main.py')
python_executable = sys.executable
num_cores = 48

In [23]:
logs_dir = os.path.abspath('../logs')
src_dir = os.path.abspath('../')
# Create the logs directory if it doesn't exist
os.makedirs(logs_dir, exist_ok=True)
logs_dir

'/groups/spruston/home/moharb/DELTA_iDISCO/logs'

In [24]:
animals = [
   'ANM555600_PSD-HT_iDISCO'
    # Add more animal IDs here if needed
]

In [25]:
for animal in animals:
    # Construct the log file paths
    output_log = os.path.join(logs_dir, f'output_{animal}.log')
    error_log = os.path.join(logs_dir, f'error_{animal}.log')

    # Construct the bsub command
    # bsub_command = [
    #     'bsub',
    #     '-J', f'job_{animal}',  # Job name
    #     '-n', str(num_cores),  # Number of CPU cores
    #     '-o', output_log,  # Output log
    #     '-e', error_log,  # Error log
    #     python_executable,
    #     python_script, 
    #     '--animal', animal,
    #     '--base_dir', base_dir,
    #     '--fx', fx,
    #     '--param_files_dir', param_files_dir,
    #     '--annotation_np', annotation_np
    # ]
    bsub_command = [
        'bsub',
        '-J', f'job_{animal}',  # Job name
        '-n', str(num_cores),  # Number of CPU cores
        '-o', output_log,  # Output log
        '-e', error_log,  # Error log
        'bash', '-c',  # Run the following as a bash command
        f"cd {src_dir} && {python_executable} {python_script} "
        f"--animal {animal} --base_dir {base_dir} --fx {fx} "
        f"--param_files_dir {param_files_dir} --annotation_np {annotation_np}"
    ]
    
    

    # Submit the job to the cluster
    print(f"Submitting job for animal: {animal}")
    print(bsub_command)
    subprocess.run(bsub_command)

Submitting job for animal: ANM555600_PSD-HT_iDISCO
['bsub', '-J', 'job_ANM555600_PSD-HT_iDISCO', '-n', '48', '-o', '/groups/spruston/home/moharb/DELTA_iDISCO/logs/output_ANM555600_PSD-HT_iDISCO.log', '-e', '/groups/spruston/home/moharb/DELTA_iDISCO/logs/error_ANM555600_PSD-HT_iDISCO.log', 'bash', '-c', 'cd /groups/spruston/home/moharb/DELTA_iDISCO/src && /groups/spruston/home/moharb/mambaforge/envs/pyants/bin/python /groups/spruston/home/moharb/DELTA_iDISCO/src/main.py --animal ANM555600_PSD-HT_iDISCO --base_dir /nrs/spruston/Boaz/I2/2024-09-09_Compare_iDISCO_EZCleat_THF --fx /nrs/spruston/Boaz/I2/atlas10_hemi.tif --param_files_dir /nrs/spruston/Boaz/I2/itk --annotation_np /nrs/spruston/Boaz/I2/annotatin10_hemi.tif']
Job <141413662> is submitted to default queue <local>.


This job will be billed to spruston
